In [36]:
# Параметры
TASK = ['max', 'max', 'min'] # Задача оптимизации
FILE_PATH = './Other/aceton729.csv'  # Откуда брать файл
PRINT_PROGRESS = False  # Вывод прогресса
SAVE_RESULTS = False  # Сохранение результата


In [35]:
# Получение данных
import pandas as pd

data_source = pd.read_csv(FILE_PATH, sep=',', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(axis=1, how='all')
print('Размер таблицы:', data_source.shape)

# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой
column_names = data_source.columns.values  # Получение названий столбцов

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float)
criteria2_column = data_source.iloc[:, 7].astype(float)
criteria3_column = data_source.iloc[:, 8].astype(float)


ERROR_VALUE = -32767
if data_source[data_source == ERROR_VALUE].count().sum() > 0 or data_source.isna().sum().sum() > 0:
    print('В таблице есть битые данные')
    print(data_source[data_source == ERROR_VALUE].count())
    print(data_source.isna().sum())


Размер таблицы: (730, 9)


In [38]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

# Псевдовеса,если нужно максимизировать критерий, то умножаем его на -1
weight = [-1 if i == 'max' else 1 for i in TASK]

# Создание интерполяторов
print('Расчет интерполятора:', column_names[6])
criteria1 = LinearNDInterpolator(points, criteria1_column.values * weight[0])
print('Расчет интерполятора:', column_names[7])
criteria2 = LinearNDInterpolator(
    points, criteria2_column.values * weight[1])
print('Расчет интерполятора:', column_names[8])
criteria3 = LinearNDInterpolator(
    points, criteria3_column.values * weight[2])
print('Расчет интерполяторов завершен')

test_point = params.sample(1).values[0]
# Главное, чтобы nan не выдавало
print('Проверка работы интерполяторов в точке', test_point)
print(column_names[6], test_point, criteria1(test_point))
print(column_names[7], test_point, criteria2(test_point))
print(column_names[8], test_point, criteria3(test_point))

# Границы параметров
lower_bound = params.min().to_numpy()
upper_bound = params.max().to_numpy()
# Количество шагов (может быть некорректно)
steps = [params[col].unique().size for col in params]

bounds = pd.DataFrame([lower_bound,upper_bound,steps],
                      columns=column_names[0:6],
                      index=['Нижняя граница', 'Верхняя граница', 'Количество шагов (возможно некорректно)'])
bounds.transpose()


Расчет интерполятора: AC____KMOL/HR
Расчет интерполятора: PROP____KMOL/HR
Расчет интерполятора: QQ____KJ/HR
Расчет интерполяторов завершен
Проверка работы интерполяторов в точке [256.85  36.85  86.85  56.85   3.5  100.  ]
AC____KMOL/HR [256.85  36.85  86.85  56.85   3.5  100.  ] [-71.3865721]
PROP____KMOL/HR [256.85  36.85  86.85  56.85   3.5  100.  ] [-535.498336]
QQ____KJ/HR [256.85  36.85  86.85  56.85   3.5  100.  ] [9.81700724e+08]


,Нижняя граница,Верхняя граница,Количество шагов (возможно некорректно)
VARY1_E-100_PARAM_TEMP_C,256.85,296.85,3.0
VARY2_E-101_PARAM_TEMP_C,-3.15,36.85,3.0
VARY3_E-102_PARAM _TEMP_C,66.85,86.85,4.0
VARY4_V-101_PARAM_TEMP_C,-23.15,56.85,3.0
VARY5_P-100_PARAM_PRES_BAR,2.00,5.00,4.0
VARY6_TEE-100_18_MOLE-FLOFLOW _KMOL/HR,100.00,160.00,4.0


In [39]:
# Однокритериальная оптимизация

import pandas as pd
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
# Алгоритмы
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.algorithms.soo.nonconvex.de import DE # Алгоритм дифференциальной эволюции, если Хука-Дживса выдает nan


class Find_best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria1(x)


class Find_best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Find_best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)

soo_c1 = minimize(Find_best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
soo_c2 = minimize(Find_best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
soo_c3 = minimize(Find_best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)

# Вывод в виде таблицы
single_criteria_optimization_df = pd.DataFrame(
    columns=column_names[0:6],
    data=[soo_c1.X, soo_c2.X, soo_c3.X],
    index=column_names[6:9]
)
single_criteria_optimization_df['Значение критерия'] = [
    weight[0] * soo_c1.F[0], weight[1] * soo_c2.F[0], weight[2] * soo_c3.F[0]]

single_criteria_optimization_df['Задание'] = TASK

print('Результаты однокритериальной оптимизации:')
single_criteria_optimization_df.transpose()


Результаты однокритериальной оптимизации:


,AC____KMOL/HR,PROP____KMOL/HR,QQ____KJ/HR
VARY1_E-100_PARAM_TEMP_C,296.85,256.85,256.85
VARY2_E-101_PARAM_TEMP_C,31.85,-3.15,36.85
VARY3_E-102_PARAM _TEMP_C,66.85,66.85,66.85
VARY4_V-101_PARAM_TEMP_C,-23.15,-23.15,56.85
VARY5_P-100_PARAM_PRES_BAR,4.625,3.5,5.0
VARY6_TEE-100_18_MOLE-FLOFLOW _KMOL/HR,100.0,160.0,100.0
Значение критерия,77.070603,539.240432,977223588.0
Задание,max,max,min


In [40]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
# Алгоритм дифференциальной эволюции
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np


def marg_objective_function(x):
    '''Маргинальная целевая функция'''
    diff_c1 = soo_c1.F - criteria1(x)
    diff_c2 = soo_c2.F - criteria2(x)
    diff_c3 = soo_c3.F - criteria3(x)

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Marg_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=lower_bound, xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = marg_objective_function(x)


# Запуск оптимизации
try:
    res_PS = minimize(
        Marg_optimization(), algorithm=PatternSearch(), seed=1)

    # Вывод результатов в виде таблицы
    margin_optimization_df = pd.DataFrame(
        data=(
            res_PS.X.tolist() + (weight[0] * criteria1(res_PS.X)).tolist() + (weight[1] * criteria2(res_PS.X)).tolist() + (weight[2] * criteria3(res_PS.X)).tolist()),
        index=column_names[0:9],
    )
except Exception as e:
    print('Ошибка маргинальной оптимизации')
    print(e)

print('Результаты маргинальной оптимизации')
margin_optimization_df


Результаты маргинальной оптимизации


,0
VARY1_E-100_PARAM_TEMP_C,2.568500e+02
VARY2_E-101_PARAM_TEMP_C,3.685000e+01
VARY3_E-102_PARAM _TEMP_C,6.685000e+01
VARY4_V-101_PARAM_TEMP_C,5.685000e+01
VARY5_P-100_PARAM_PRES_BAR,5.000000e+00
VARY6_TEE-100_18_MOLE-FLOFLOW _KMOL/HR,1.000000e+02
AC____KMOL/HR,7.138460e+01
PROP____KMOL/HR,5.354986e+02
QQ____KJ/HR,9.772236e+08


In [41]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA


# Постановка задачи
class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=lower_bound,
                         # Максимальные значения параметров
                         xu=upper_bound)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS)  # Вывод прогресса)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['c1'] = weight[0] * res_NSGA2.F[:, 0]
    res_NSGA2_df['c2'] = weight[1] * res_NSGA2.F[:, 1]
    res_NSGA2_df['c3'] = weight[2] * res_NSGA2.F[:, 2]
    res_NSGA2_df.columns = column_names

    if SAVE_RESULTS:
        res_NSGA2_df.to_csv('NSGA2_Results.csv', sep=',', index=False)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS,
                           termination=('time', '00:01:00') # Не позволяем наглеть
                           )

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['c1'] = weight[0] * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['c2'] = weight[1] * res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['c3'] = weight[2] * res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.columns = column_names

    if SAVE_RESULTS:
        res_AGEMOEA_df.to_csv('AGEMOEA_Results.csv', sep=',', index=False)
        
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)



Многокритериальная оптимизация (Метод NSGA2)
Многокритериальная оптимизация (Метод AGEMOEA)


In [42]:
res_NSGA2_df

,VARY1_E-100_PARAM_TEMP_C,VARY2_E-101_PARAM_TEMP_C,VARY3_E-102_PARAM _TEMP_C,VARY4_V-101_PARAM_TEMP_C,VARY5_P-100_PARAM_PRES_BAR,VARY6_TEE-100_18_MOLE-FLOFLOW _KMOL/HR,AC____KMOL/HR,PROP____KMOL/HR,QQ____KJ/HR
0,256.850019,-3.149998,66.850002,-23.150000,3.501626,160.000000,77.056370,539.240432,1.004118e+09
1,256.850450,-3.150000,66.850000,56.850000,2.000042,100.000006,71.378554,535.497359,9.766391e+08
2,296.824806,16.571588,86.836704,-23.137987,2.031549,159.999303,77.070024,539.226796,1.012378e+09
3,256.850000,-3.150000,66.850000,56.850000,2.000000,100.000000,71.378554,535.497359,9.766390e+08
4,296.850000,6.164253,82.196113,-23.149996,2.698407,100.000018,77.073026,539.213849,1.003230e+09
...,...,...,...,...,...,...,...,...,...
95,256.854908,-3.148152,66.850175,33.621708,2.004409,100.000010,74.501026,537.662174,9.832271e+08
96,256.853963,-3.147174,66.850000,44.820335,2.192806,100.120651,72.995828,536.618702,9.801530e+08
97,256.857851,-3.004325,66.851428,37.731419,2.001738,100.004046,73.948644,537.279235,9.820647e+08
98,256.858852,-3.112598,66.850003,55.897901,2.005271,100.004651,71.506606,535.586179,9.769140e+08


In [43]:
res_AGEMOEA_df

,VARY1_E-100_PARAM_TEMP_C,VARY2_E-101_PARAM_TEMP_C,VARY3_E-102_PARAM _TEMP_C,VARY4_V-101_PARAM_TEMP_C,VARY5_P-100_PARAM_PRES_BAR,VARY6_TEE-100_18_MOLE-FLOFLOW _KMOL/HR,AC____KMOL/HR,PROP____KMOL/HR,QQ____KJ/HR
0,256.850007,-3.149796,66.850109,51.606097,2.000001,100.000155,72.083468,535.986077,9.781258e+08
1,256.850026,-3.149714,66.850357,50.136541,2.000006,100.000100,72.281013,536.123036,9.785425e+08
2,256.850367,-3.149981,66.850005,18.873353,2.000014,100.000106,76.483579,539.036680,9.874057e+08
3,256.850012,-3.149962,66.850000,56.849993,2.000001,100.000033,71.378555,535.497360,9.766390e+08
4,256.852252,-3.145019,66.850030,30.578524,2.000129,100.000036,74.910108,537.945790,9.840877e+08
...,...,...,...,...,...,...,...,...,...
95,256.850911,-3.149997,66.850001,36.361792,2.000002,100.000002,74.132689,537.406804,9.824477e+08
96,256.850071,-2.691818,66.857325,43.541521,2.000033,100.000106,73.167591,536.737686,9.804217e+08
97,296.036706,-3.056897,66.852902,-23.144016,2.292290,146.717755,77.066577,539.224682,1.005983e+09
98,256.856956,-1.569652,66.851648,41.227621,2.000156,100.000127,73.478735,536.953358,9.810961e+08
